<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/loading_and_saving_configs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [ ]:
!pip install ipython ipykernel --upgrade

# Set Up Environment

### Import required libraries

In [ ]:
!pip install -q canopy

In [ ]:
import canopy
import logging
import nest_asyncio

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

### Authenticate

In [ ]:
authentication_data = canopy.prompt_for_authentication()
session = canopy.Session(authentication_data)

# Set Up Example

In [ ]:
# Removes sensitive data from configs so we don't publish them in our public repo.
def sanitize_config(config: canopy.ConfigResult):
    config.document.tenant_id = \
    config.document.user_id = \
    'removed'
    
    for t in config.user_information.tenants:
        t.tenant_id = \
        t.name = \
        t.short_name = \
        'removed'

        for u in t.users:
            u.user_id = \
            u.username = \
            u.email = \
            'removed'

# Example: Loading and saving configs

## User configs and default configs
Configs on the platform are split into user configs, which are created by users, and default configs which are supplied by Canopy. When viewing lists of configs in the Canopy Portal (the web UI), the user configs appear first and the default configs are in a separate list below.

When using the Canopy Python library, there are separate functions for loading user and default configs.

## Loading a default config

In [ ]:
default_weather = await canopy.load_default_config(session, 'weather', '25 deg, dry')
default_car = await canopy.load_default_config(session, 'car', 'Canopy F1 Car 2019')

print(default_weather)

{
  "config_type": "weather",
  "name": "25 deg, dry",
  "properties": null,
  "notes": null,
  "user_id": null,
  "config_id": null,
  "is_edited": false,
  "is_data_converted": false,
  "data": {
    "TAir": 25,
    "rHumidityRelative": 0.1,
    "pAirAtmosphericLocal": 101300
  }
}


## Advanced Usage: The `data` vs `raw_data` properties.

The `default_weather` object has two properties for accessing the config data: `default_weather.data` and `default_weather.raw_data`.

The reason for this is a compromise between usability and performance. By default, Python deserialises the config data into nested dictionaries. These must be accessed with indexers:

In [ ]:
default_weather.raw_data['TAir']

25

In [ ]:
default_car.raw_data['suspension']['front']['internal']['damper']['dampingCoefficient']

1200

It would be much nicer to be able to traverse the hierarchy as if it were a set of nested properties. This is what the `data` property allows you to do.

When you access the `data` property for the first time it converts the `raw_data` from nested dictionaries to a structure supporting property traversal, allowing you to access the data more concisely:

In [ ]:
default_weather.data.TAir

25

In [ ]:
default_car.data.suspension.front.internal.damper.dampingCoefficient

1200

Once this conversion has happened, the `raw_data` property changes to output the same as the `data` property. This is so that if you make any modifications they appear on both properties.

In [ ]:
default_car.data.suspension.front.internal.damper.dampingCoefficient = 1300

print(default_car.data.suspension.front.internal.damper.dampingCoefficient)
print(default_car.raw_data['suspension']['front']['internal']['damper']['dampingCoefficient'])

# This next line would result in an error if the data property had not already been accessed:
print(default_car.raw_data.suspension.front.internal.damper.dampingCoefficient)

print(default_car.is_data_converted)

1300
1300
1300
True


Why not always perform the conversion, and just have one property? If the config is particularly large then the conversion is not cheap. 

If you have only downloaded a config in order to pass it into a study, performing the conversion would result in unnecessary delay with no benefit. If you are in a performance critical bit of code, you may similarly prefer to use the dictionary notation and avoid the conversion.

## Saving a user config

Before we can show how to load user configs, we should create one:

In [ ]:
user_weather_id_july = await canopy.create_config(
    session,
    'weather',
    'Python Example Weather July',
    default_weather.raw_data,
    properties={ 'country':'uk', 'month':'july' })

default_weather.data.TAir = 10

user_weather_id_march = await canopy.create_config(
    session,
    'weather',
    'Python Example Weather March',
    default_weather.raw_data,
    properties={ 'country':'uk', 'month':'march' })
    
print(f'July ID: {user_weather_id_july}')
print(f'March ID: {user_weather_id_march}')

July ID: a2d1d73bde0841639b9f25574d5644c3
March ID: 822bc12a3df54bf79b87fd84f3731efe


## Loading a user config by ID

The first way to load a config is by its ID:

In [ ]:
march_weather = await canopy.load_config(session, user_weather_id_march)

sanitize_config(march_weather)

print(march_weather)

{
  "config_id": "822bc12a3df54bf79b87fd84f3731efe",
  "document": {
    "document_id": "822bc12a3df54bf79b87fd84f3731efe",
    "tenant_id": "removed",
    "user_id": "removed",
    "name": "Python Example Weather March",
    "type": "config",
    "sub_type": "weather",
    "sim_version": "1.3371",
    "creation_date": "2020-04-27 16:36:57.562479+00:00",
    "modified_date": "2020-04-27 16:36:57.562479+00:00",
    "properties": {
      "country": "uk",
      "month": "march"
    },
    "data": {
      "TAir": 10,
      "rHumidityRelative": 0.1,
      "pAirAtmosphericLocal": 101300
    },
    "support_session": null,
    "notes": null,
    "delete_requested": null,
    "parent_worksheet_id": null
  },
  "user_information": {
    "tenants": [
      {
        "tenant_id": "removed",
        "name": "removed",
        "short_name": "removed",
        "users": [
          {
            "user_id": "removed",
            "username": "removed",
            "email": "removed"
          }
      

This gives us a lot more information than when we loaded the default config. For example the `user_information` section contains the resolved tenant and user information for the document owner, or anyone who has contributed to the support session.

If you don't need all this information, we can convert it to the same `canopy.LocalConfig` structure which the default config used:

In [ ]:
march_weather_2 = march_weather.to_local_config()

print(march_weather_2)

{
  "config_type": "weather",
  "name": "Python Example Weather March",
  "properties": {
    "country": "uk",
    "month": "march"
  },
  "notes": null,
  "user_id": "removed",
  "config_id": "822bc12a3df54bf79b87fd84f3731efe",
  "is_edited": false,
  "is_data_converted": false,
  "data": {
    "TAir": 10,
    "rHumidityRelative": 0.1,
    "pAirAtmosphericLocal": 101300
  }
}


This simpler form can be useful if we want all our functions to use the same config data structure.

## Getting a Config ID from the Canopy Portal

If you want to load an existing config by ID, you can find its ID by opening it in the Canopy Portal (the web UI to the platform).

The URL for a config is of the form `/configs/<configType>/<tenantId>/<configId>/edit`. You can therefore simply copy the Config ID from the URL and paste it into your Python code.

Similarly for studies the URL is of the form `/studies/<tenantId>/<studyId>`.

Failing that, you can load the config by name or other metadata, which we'll do next.


## Loading a user config by name / metadata

Sometimes it is more convenient to load a config by name:

In [ ]:
july_weather_by_name = await canopy.find_config(
    session, 
    'weather', 
    name='Python Example Weather July')

march_weather_by_name = await canopy.find_config(
    session,
    'weather',
    name='Python Example Weather March')

print(july_weather_by_name.data.TAir)
print(march_weather_by_name.data.TAir)

25
10


We can also search by other metadata, such as custom properties:

In [ ]:
july_weather_by_property = await canopy.find_config(
    session,
    'weather',
    custom_properties={ 'month': 'july' })

march_weather_by_property = await canopy.find_config(
    session,
    'weather',
    custom_properties={ 'month': 'march', 'country': 'uk' })

print(july_weather_by_name.data.TAir)
print(march_weather_by_name.data.TAir)

25
10


When multiple properties match the criteria, it will return the most recently modified:

In [ ]:
most_recently_modified_uk_weather = await canopy.find_config(
    session,
    'weather',
    custom_properties={ 'country': 'uk' })

print(most_recently_modified_uk_weather.data.TAir)

10


Finally, if your config was in a worksheet, you can also pass in a `parent_worksheet_id` parameter to find the config within the worksheet.